In [45]:
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import os
import gc
import utilities
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN

In [4]:
def get_folder_names(path):
    list_of_files = {}
    for (dirpath, dirnames, filenames) in os.walk(path):
        return  dirnames

In [7]:
data = utilities.load_train_data('./train')

# with open("tpm_train.pkl", 'rb') as input_file:
#     data = pkl.load(input_file)
    

(369L, 199326L)


In [8]:
type(data)

numpy.ndarray

In [9]:
data = pd.DataFrame(data)

In [10]:
data.set_index(0, inplace=True)
data.head()

,1,2,3,4,5,6,7,8,9,10,...,199316,199317,199318,199319,199320,199321,199322,199323,199324,199325
0,,,,,,,,,,,,,,,,,,,,,
ERR188021,0.221792,0.0,5.4206,0.0,0.0,0.0,0.0,0.0,1.08004,0.0,...,35.2899,36.7604,92.426,3532.69,5837.02,299.965,3237.99,85.9032,2026.97,TSI
ERR188022,0.797727,0.0,4.58229,0.0,0.0,0.0,0.0,0.0,1.68431,0.0,...,17.225,22.2112,83.2541,3149.23,6786.43,88.9958,3629.21,437.084,2652.65,CEU
ERR188023,0.37259,0.0,4.22841,0.375841,0.0,0.0,0.0,0.0,0.0383236,0.0,...,57.5201,90.9536,171.122,2886.24,6365.47,227.204,4590.79,353.977,1974.74,FIN
ERR188027,0.211032,0.0,2.53578,0.0,0.0,0.0,0.0,0.0,1.65247,0.0,...,74.8504,51.6985,306.338,3710.86,10669.3,283.218,4681.41,813.604,3703.72,CEU
ERR188029,0.08016,0.0,3.88313,0.0,0.0,0.0,0.0,0.0,0.594998,0.0,...,20.523,42.6877,103.764,2686.65,5672.71,136.82,3866.59,275.444,1796.54,FIN


In [11]:
label = data[199325]
label.head()

0
ERR188021    TSI
ERR188022    CEU
ERR188023    FIN
ERR188027    CEU
ERR188029    FIN
Name: 199325, dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data.drop([199325], axis=1), label, test_size=0.3)

In [13]:
clf = RandomForestClassifier(n_estimators=200, max_depth=60, random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print f1_score(y_test, y_pred, average='macro')


0.80310782241


In [17]:
# clf = LinearSVC(random_state=0)
scores = cross_val_score(clf, data.drop([199325], axis=1), label, cv=5,scoring='accuracy')
# clf.fit(X_train, y_train)


# print clf.coef_

# print clf.intercept_

# y_pred = clf.predict(X_test)c

# print f1_score(y_test, y_pred, average='macro')
print scores.mean(), scores.std()

0.819011544012 0.037116215971


metric    cv    mean_score     std <br>
f1 macro  5      0.813775    0.041139<br>
accuracy  5      0.819011    0.037116<br>

In [50]:
svc = SVC(C=0.01)
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)

print f1_score(y_test, y_pred, average='macro')
print accuracy_score(y_test,y_pred), accuracy_score(y_train,svc.predict(X_train))

0.0584615384615
0.171171171171 0.22480620155


In [53]:
knn = KNN(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

print f1_score(y_test, y_pred, average='macro')
print accuracy_score(y_test,y_pred), accuracy_score(y_train,knn.predict(X_train))

0.344101506836
0.351351351351 0.662790697674


In [6]:
folder_names = get_folder_names("./train/")

master_set = set()
all_eq_c = {}
for f in folder_names:
    with open("./train/" + f +"/bias/aux_info/eq_classes.txt") as eq_file:
        eq_c = eq_file.read()
    eq_c = eq_c.split("\n")
    eq_c = eq_c[199326:]
    cur_eq_c = {}
    
    for e in eq_c:
        e = e.split('\t')
        cur_eq_c[''.join(e[:-1])] = e[-1]
        master_set.add(''.join(e[:-1]))
    
    
    all_eq_c[f] = cur_eq_c
    

In [20]:
# pkl.dump(master_set,open('master_set.pkl','wb'))
# pkl.dump(all_eq_c,open('all_eq_c.pkl','wb'))
for f in folder_names:
    with open('./all_equi_classes/'+f+'.pkl','wb') as filename:
        pkl.dump(all_eq_c[f],filename)

In [21]:
del all_eq_c
gc.collect()

7

In [ ]:
with open('master_set.pkl','rb') as ms:
    master_set = pkl.load(ms)

col_list = ['sample_name'] +list(master_set)[1:]

In [27]:
train = pd.DataFrame(np.zeros((len(folder_names),len(master_set))),index=folder_names,columns= col_list)

del master_set
gc.collect()

16

In [39]:
for f in folder_names:
    with open('./all_equi_classes/'+f+'.pkl','rb') as filename:
        ec = pkl.load(filename)
    for e in ec:
        if e == '':
            continue
        train[e][f] = ec[e]

In [44]:
# with open('train_equi.pkl','wb') as filename:
#         pkl.dump(train,filename)

In [45]:
train_copy = train.copy()

In [48]:
# train.describe()

,sample_name,1133067,1057217572185722057224572255722657227572365723757238,62891628942689757315593793149528,11106187106188106189106190106191106192106193106194106195106196106197,4130829130831130833130834,21179193776738915431484314943150452094706749275670846709177013120062144584155596184433187893188800190330194852196102,2590750206,3197966197968197969,26198261983,...,256592107323,3802518025280253,2120089194639,24043549006,10128265128266128267128281128282128283128285128286128288128291,270313195682,3108298108301108303,11142360142367142369142370142371142372142373142375142376142378142379,6169536169538169539169540169542169544,81235972293722957229772298722997230072302
count,369.0,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,...,369.000000,369.000000,369.000000,369.000000,369.00000,369.000000,369.000000,369.000000,369.000000,369.000000
mean,0.0,1.647696,0.121951,0.043360,94.094851,0.523035,1.891599,0.040650,57.284553,1.371274,...,0.002710,0.159892,0.027100,0.008130,0.03252,0.010840,1.626016,0.073171,1.775068,0.024390
std,0.0,1.511011,0.359318,0.203944,65.292229,0.776794,1.720768,0.197747,40.743535,1.455999,...,0.052058,0.447111,0.162596,0.089922,0.19231,0.103691,1.847729,0.290354,2.164643,0.171157
min,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000,45.000000,0.000000,1.000000,0.000000,27.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,1.000000,0.000000,0.000000,83.000000,0.000000,2.000000,0.000000,46.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,0.0,2.000000,0.000000,0.000000,135.000000,1.000000,3.000000,0.000000,81.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,2.000000,0.000000,3.000000,0.000000
max,0.0,9.000000,2.000000,1.000000,386.000000,4.000000,9.000000,1.000000,257.000000,8.000000,...,1.000000,3.000000,1.000000,1.000000,2.00000,1.000000,11.000000,3.000000,12.000000,2.000000


In [80]:
# train.keys()
tr_lab = pd.read_csv('p1_train.csv')
merged = pd.merge(train,tr_lab,left_on='sample_name',right_on='accession')

In [81]:
merged.head()

,sample_name,1133067,1057217572185722057224572255722657227572365723757238,62891628942689757315593793149528,11106187106188106189106190106191106192106193106194106195106196106197,4130829130831130833130834,21179193776738915431484314943150452094706749275670846709177013120062144584155596184433187893188800190330194852196102,2590750206,3197966197968197969,26198261983,...,2120089194639,24043549006,10128265128266128267128281128282128283128285128286128288128291,270313195682,3108298108301108303,11142360142367142369142370142371142372142373142375142376142378142379,6169536169538169539169540169542169544,81235972293722957229772298722997230072302,accession,label
0,ERR188021,5.0,0.0,0.0,184.0,0.0,3.0,0.0,122.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,ERR188021,TSI
1,ERR188022,2.0,0.0,0.0,58.0,2.0,0.0,0.0,33.0,5.0,...,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,ERR188022,CEU
2,ERR188023,3.0,0.0,0.0,135.0,0.0,2.0,0.0,42.0,1.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,ERR188023,FIN
3,ERR188027,4.0,0.0,1.0,195.0,0.0,1.0,0.0,27.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,ERR188027,CEU
4,ERR188029,1.0,1.0,0.0,34.0,0.0,6.0,0.0,88.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,ERR188029,FIN


In [66]:
# train['sample_names'].head()

ERR188021    ERR188021
ERR188022    ERR188022
ERR188023    ERR188023
ERR188027    ERR188027
ERR188029    ERR188029
Name: sample_names, dtype: object

In [1]:
tr_lab[tr_lab['accession'] == 'ERR188023']

NameError: name 'tr_lab' is not defined

In [ ]:
tr

In [83]:
label = merged['label']

X_train, X_test, y_train, y_test = train_test_split(merged.drop(['sample_name','accession','label'], axis=1), label, test_size=0.2)

In [2]:
gc.co

NameError: name 'f' is not defined